In [2]:
import json,os,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath("..")) 
from transformers import AutoTokenizer
import numpy as np
from utils.utils import seed_all

seed_all()

In [3]:
with open("../aside/data/train_data/toolace_train_parsed.json", "r") as f:
    data = json.load(f)
print (len(data))

10141


In [ ]:
new_data = []
valid_assistant_count = 0
valid_tool_count = 0
tool_exists_count = 0
for d in data: # we mainly want to train on assistant chats that pick tool and not just respond and we only want to do it if there is already a tool use before that
    valid_assistant_turns = []
    tool_available_turns = []
    if any([turn['from'] == 'tool' for turn in d["conversations"]]):
        tool_exists_count += 1
    for j,turn in enumerate(d["conversations"]):
        if turn['from'] == 'tool': # check if next one is assistant and contains tool use, if yes record hte idx down
            if j+1 < len(d["conversations"]) and d["conversations"][j+1]["from"] == "assistant" and "<tool_call>" in d["conversations"][j+1]["value"]:
                valid_assistant_turns.append(j+1)
            elif j+1 < len(d["conversations"]) and d["conversations"][j+1]["from"] == "assistant": # if not tool use but is assistant, record it as well
                tool_available_turns.append(j+1)
    if len(valid_assistant_turns) > 0:
        valid_assistant_count += 1
        random_idx = np.random.choice(valid_assistant_turns)
    elif len(tool_available_turns) > 0:
        valid_tool_count += 1
        random_idx = np.random.choice(tool_available_turns)
    else:
        continue
    new_conversations = d["conversations"][:random_idx+1] # take until that turn, the last assistant will be the one that the model needs to predict
    d_copy = d.copy()
    d_copy["conversations"] = new_conversations
    new_data.append(d_copy)

print (len(new_data), valid_assistant_count, valid_tool_count)
            
            
            

742 273 469 742
